# One-hotエンコーディング
One-hotエンコーディングを行うことでテキストをベクトル化することができる. One-hotエンコーディングによるテキスト表現は直観的で実装が簡単であるが, ベクトルの大きさが語彙数に比例することによってスパースな表現になる, テキストを固定長で表現できない, 単語間の類似性という概念を持たない, 未知の単語に適応できないという問題点がある.

In [8]:
documents = ["Dog bites man.", "Man bites dog.", "Dog eats meat.", "Man eats food."]
# 小文字化してピリオドを取り除く
processed_docs = [doc.lower().replace(".","") for doc in documents]

# vocabの構築
vocab={}
count = 0
for doc in processed_docs:
    for word in doc.split():
        if word not in vocab:
            count = count+1
            vocab[word] = count
print(vocab)

def get_onehot_vector(somestring):
    onehot_encoded = []
    for word in somestring.split():
        temp = [0]*len(vocab)
        if word in vocab:
            temp[vocab[word]-1] = 1
        onehot_encoded.append(temp)
    return onehot_encoded

print(processed_docs[1])
print(get_onehot_vector(processed_docs[1]))

{'dog': 1, 'bites': 2, 'man': 3, 'eats': 4, 'meat': 5, 'food': 6}
man bites dog
[[0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]]


## Bag of Words
Bag of Wordsはテキストを単語の集合として表現することでテキスト表現を行う手法である. BoWは同じ単語を含むテキストのベクトル表現が近くなるため文書の類似性を捉えているといえる. また任意の長さの文を固定長の符号で表すことができる. 一方でスパース性の問題や, 同じ意味をもつ異なる単語への類似性がないこと, OOVが処理できない, 語順が失われるという問題がある.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
bow_rep = count_vect.fit_transform(processed_docs)

# 語彙を表示
print(count_vect.vocabulary_)

# Bowを表示
for i in range(len(processed_docs)):
    print(processed_docs[i])
    print(bow_rep[i].toarray())
    
# 新しいテキストに対するBoW
temp = count_vect.transform(["dog and dog are friends"])
print(temp.toarray())

{'dog': 1, 'bites': 0, 'man': 4, 'eats': 2, 'meat': 5, 'food': 3}
dog bites man
[[1 1 0 0 1 0]]
man bites dog
[[1 1 0 0 1 0]]
dog eats meat
[[0 1 1 0 0 1]]
man eats food
[[0 0 1 1 1 0]]
[[0 2 0 0 0 0]]


## Bag of N-grams
Bag of N-grams(BoN)はテキストを連続するn個の単語に分割することでフレーズや語順を考慮したテキスト表現を作成する方法である. BoNは同じnグラムを含む文書に対する類似性を捉えることができる一方で, nが増加するとスパース性が急速に増加する, OOVが処理できないという問題がある.

In [13]:
# n=1,2,3のときのBoN
count_vect = CountVectorizer(ngram_range=(1,3))

bow_rep = count_vect.fit_transform(processed_docs)
# 語彙を表示
print(count_vect.vocabulary_)


{'dog': 3, 'bites': 0, 'man': 12, 'dog bites': 4, 'bites man': 2, 'dog bites man': 5, 'man bites': 13, 'bites dog': 1, 'man bites dog': 14, 'eats': 8, 'meat': 17, 'dog eats': 6, 'eats meat': 10, 'dog eats meat': 7, 'food': 11, 'man eats': 15, 'eats food': 9, 'man eats food': 16}


In [15]:
# 新しいテキストに対するBoW

# Bowを表示
for i in range(len(processed_docs)):
    print(processed_docs[i])
    print(bow_rep[i].toarray())

temp = count_vect.transform(["dog and dog are friends"])
print(temp.toarray())

dog bites man
[[1 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0]]
man bites dog
[[1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0]]
dog eats meat
[[0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1]]
man eats food
[[0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0]]
[[0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
